# Utility functions

In [ ]:
import os
from math import sqrt
import numpy as np
import cv2
import matplotlib
import matplotlib.pyplot as plt
from PIL import Image
from scipy.signal import convolve2d
from scipy.ndimage import gaussian_filter
from scipy.signal import medfilt

In [ ]:
def wpsnr(img1, img2):
    img1 = np.float32(img1)/255.0
    img2 = np.float32(img2)/255.0
    difference = img1-img2
    same = not np.any(difference)
    if same is True:
        return 9999999
    w = np.genfromtxt('csf.csv', delimiter=',')
    ew = convolve2d(difference, np.rot90(w,2), mode='valid')
    decibels = 20.0*np.log10(1.0/sqrt(np.mean(np.mean(ew**2))))
    return decibels

## Edges detection

In [ ]:
def sobel_edges(img):
  sobelimage=img.copy()
  sobelx = cv2.Sobel(sobelimage,cv2.CV_64F,1,0,ksize=3)
  sobely = cv2.Sobel(sobelimage,cv2.CV_64F,0,1,ksize=3)
  magnitude = np.sqrt(sobelx**2 + sobely**2)
  magnitude = cv2.convertScaleAbs(magnitude)
  _, thresholded = cv2.threshold(magnitude, 100, 255, cv2.THRESH_BINARY)
  return thresholded
  
def canny_edges(img):
  th1=30
  th2=60
  d=2
  edgeresult=img.copy()
  edgeresult = cv2.GaussianBlur(edgeresult, (2*d+1, 2*d+1), -1)[d:-d,d:-d]
  edges = cv2.Canny(edgeresult, th1, th2)
  return edges

## Basic attack functions

In [ ]:
def blur(img, sigma):
  attacked = gaussian_filter(img, sigma)
  return attacked

def median(img, sigma):
  attacked = medfilt(img, sigma)
  return attacked

def awgn(img, std, seed):
  mean = 0.0
  np.random.seed(seed)
  attacked = img + np.random.normal(mean, std, img.shape)
  attacked = np.clip(attacked, 0, 255)
  return attacked

# Edge-preserving blur

In [ ]:
def edge_preserving_blur(img, sigma, edge_func, blur_func):
  edges = edge_func(img)
  edges = edges / edges.max()
  edges = cv2.resize(edges, (img.shape[1], img.shape[0]))
  blurred_img = blur_func(img, sigma)
  attacked = edges * img + (1 - edges) * blurred_img
  return attacked

# AWGN on edges

In [ ]:
def edge_awgn(img, std, seed, edge_func):
    attacked = img.copy()
    edges_results = img.copy()
    global_awgn = awgn(img, std, seed)
    edges = edge_func(img)
    edges_results[edges > 0] = [255]
    attacked[edges > 0] = global_awgn[edges > 0] 
    return attacked